In [ ]:
from flask import Flask, jsonify
import psycopg2
from psycopg2.extras import RealDictCursor
from config import db_name, user, password, host

app = Flask(__name__)

def get_db_connection():
    conn = psycopg2.connect(
            dbname=db_name,
            user=user,
            password=password,
            host=host
        )
    return conn

@app.route('/map', methods=['GET'])
def get_landslide_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('''
    SELECT
            cod_reg,
            cod_rip,
            COD_PROV,
            ar_kmq,
            nome,
            uid,
            ar_fr_p3p4,
            ar_fr_p2,
            ar_fr_p1,
            pop_fr_p2,
            pop_fr_p1,
            ed_fr_p2,
            ed_fr_p1,
            ar_frp3p4p,
            popfrp3p4p,
            ed_fr_p3p4,
            edfrp3p4p,
            ST_AsText(ST_SnapToGrid(geometry, 0.001)) as geom_wkt  -- Adjust 0.001 for your desired precision
        FROM dataset
    ''')
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return jsonify(data)

if __name__ == '__main__':
    app.run(debug=True, port=5000, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
